In [1]:
# ==========================================
# 0. 导入必要的库
# ==========================================

import numpy as np
import scipy.fft as fft
from scipy import ndimage
import tifffile
import matplotlib.pyplot as plt
from skimage.restoration import richardson_lucy
import os
from scipy.special import erf

In [ ]:
# ==========================================
# 1. 基础工具函数
# ==========================================

def generate_rsf(gama, n=None):
    if n is None:
        n = int(np.ceil(gama / np.sqrt(8 * np.log(2)) * np.sqrt(-2 * np.log(0.0002))) + 1)
    sigma = gama / np.sqrt(8 * np.log(2))
    kernelRadius = min(int(np.ceil(sigma * np.sqrt(-2 * np.log(0.0002))) + 1), int(np.floor(n / 2)))
    ii = np.arange(-kernelRadius, kernelRadius + 1)
    rsf_x = 0.5 * (erf((ii + 0.5) / (np.sqrt(2) * sigma)) - erf((ii - 0.5) / (np.sqrt(2) * sigma)))
    kernel = np.outer(rsf_x, rsf_x)
    return kernel / np.sum(kernel)

# generate_rsf:模拟显微镜的“眼睛”是如何看一个点的。（PSF）

🔬显微镜的成像过程可以解释为：$$ P_f = P_o * PSF +N $$
其中，$P_f$ 是最终的图像，$P_o$ 是原始图像，$PSF$ 是点扩散函数（Point Spread Function），$N$ 是噪声。

这个函数生成的二维核 $K$（即 kernel）可以拆解为两个一维向量 $x$ 和 $y$ 的外积。

## 1.核心物理模型
对于一维坐标轴上的某一个像素 $i$，其接收到的光强 $rsf\_x(i)$ 并不是简单地取中心点的值，而是对该像素范围（从 $i-0.5$ 到 $i+0.5$）进行积分：
$$ rsf\_x(i) = \int_{i-0.5}^{i+0.5} \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{t^2}{2\sigma^2}} dt $$
利用误差函数的定义 $\text{erf}(x) = \frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} dt$，我们可以将其推导为代码中的形式（完全等同）：
$$ rsf\_x(i) = \frac{1}{2} \left[ \text{erf}\left( \frac{i+0.5}{\sqrt{2}\sigma} \right) - \text{erf}\left( \frac{i-0.5}{\sqrt{2}\sigma} \right) \right] $$

## 2. 二维合成与归一化
最终的二维核 $K$ 是由横向和纵向的分布相乘得到的，并确保所有像素的光强总和为 1
$$ K = \frac{rsf\_x^T \cdot rsf\_x}{\sum (rsf\_x^T \cdot rsf\_x)} $$

## 3. 参数转换
代码中的 gama 通常指半高全宽（FWHM），它与标准差 $\sigma$ 的关系是
$$ \sigma = \frac{\text{gama}}{\sqrt{8 \ln 2}} \approx \frac{\text{gama}}{2.355} $$

In [ ]:
# ==========================================
# 2. 核心算法函数
# ==========================================

def fourier_interpolation(img, itp_fac):
    if img.ndim == 2:
        h, w = img.shape
    elif img.ndim == 3:
        frames, h, w = img.shape
    
    new_h, new_w = int(h * itp_fac), int(w * itp_fac)
    im_f = fft.fft2(img, axes=(-2, -1))
    im_f_shifted = fft.fftshift(im_f, axes=(-2, -1))
    
    pad_h, pad_w = (new_h - h) // 2, (new_w - w) // 2
    if img.ndim == 3:
        pad_width = ((0, 0), (pad_h, pad_h), (pad_w, pad_w))
    else:
        pad_width = ((pad_h, pad_h), (pad_w, pad_w))
        
    im_f_padded = np.pad(im_f_shifted, pad_width, mode='constant', constant_values=0)
    out = fft.ifft2(fft.ifftshift(im_f_padded, axes=(-2, -1)), axes=(-2, -1))
    return np.real(out) * (itp_fac ** 2)

def cumulant(im, n):
    if n == 2:
        term = im[:-1, :, :] * im[1:, :, :]
        return np.mean(term, axis=0)
    raise NotImplementedError("Currently only supports 2nd order")

def rl_core(data, kernel, iterations):
    if data.ndim == 3:
        deconvolved = np.zeros_like(data)
        for i in range(data.shape[0]):
             deconvolved[i] = richardson_lucy(data[i], kernel, num_iter=iterations, clip=False)
        return deconvolved
    return richardson_lucy(data, kernel, num_iter=iterations, clip=False)


# 
## 1. fourier_interpolation：无损放大（傅里叶插值）上采样
### 功能：在不增加新信息的情况下，让图像的像素网格变得更密。
### 核心思想：频域补零 (Zero Padding)
想象一张低分辨率的图片，它的像素很大。如果我们直接把像素拉大（线性插值），图像会变糊。傅里叶插值的做法是：把图像变成波，告诉它“在高频（细节）部分没有东西”，然后再变回图像。
### 数学推导
傅里叶变换 (FFT)：把图像 $f(x,y)$ 从空间域变到频率域 $F(u,v)$。
$$ F(u,v) = \mathcal{F}\{f(x,y)\} $$
### 代码：fft.fft2(img) 和 fft.fftshift（把低频移到中心）。
补零 (Padding)：我们在 $F(u,v)$ 的周围填上一圈 0。这相当于扩充了频谱的范围，但没有增加新的频率成分。原图大小：$ H \times W $新图大小：$H' \times W'$ (例如放大2倍)

数学操作：
$$F_{new}(u,v) = \begin{cases} F(u,v) & \text{if } (u,v) \text{ in center} \\ 0 & \text{otherwise} \end{cases} $$
代码：np.pad(im_f_shifted, ...) 就是在做这件事。

反变换 (IFFT)：把变大后的频谱 $F_{new}$ 变回空间域。
$$ f_{new}(x,y) = \mathcal{F}^{-1}\{F_{new}(u,v)\} $$
### 能量守恒修正：
FFT 的能量与采样点数 $N$ 有关。当我们把矩阵尺寸扩大了 $k$ 倍（面积扩大 $k^2$ 倍）但只填了 0，反变换回来的数值会被“稀释”（亮度减少了）。为了保持图像亮度不变，必须乘以放大倍数的平方。

## 2. cumulant：二阶累积量（提取相关性）
### 功能：
这是 SOFI (Super-resolution Optical Fluctuation Imaging) 的核心。它利用时间的涨落来去除噪声并提升分辨率。
代码翻译过来就是：取这一帧 ($t$) 和下一帧 ($t+1$)，对应像素相乘，然后取平均。
### 数学公式：
XC-SOFI (Cross-Cumulant)这就是计算时间互相关函数（在时滞 $\tau=1$ 时）：
$$ C_2(\mathbf{r}) = \langle \delta I(\mathbf{r}, t) \cdot \delta I(\mathbf{r}, t+\tau) \rangle_t $$
$I(\mathbf{r}, t)$：在位置 $\mathbf{r}$ 时间 $t$ 的光强。$\delta I$：光强的涨落（减去均值后）。$\tau$：时间延迟（$\langle \cdot \rangle_t$：对时间取平均。为什么要这样做？（核心物理意义）去噪声（重点！）：相机的电子噪声（Shot Noise）是随机的、白噪声。这一帧的噪声和下一帧的噪声没有关系（不相关）。$Noise(t) \times Noise(t+1) \approx 0$（平均后抵消）。但是！荧光分子的闪烁是有持续时间的，这一帧亮，下一帧可能还亮。$Signal(t) \times Signal(t+1) > 0$。结论：用这一行代码，完美的把背景噪声过滤掉了，只留下了真实的信号。

提升分辨率：数学证明显示，二阶累积量的点扩散函数（PSF）是原图 PSF 的平方：$U_{SOFI}(r) = U(r)^2$。高斯函数的平方，宽度变窄了 $\sqrt{2}$ 倍。宽度变窄 = 分辨率提高。
## 3. rl_core：Richardson-Lucy 反卷积
### 功能：图像复原。已知“原本清晰的图”是怎么被“模糊”的（PSF），反推清晰的图。
### 数学公式：贝叶斯迭代这是一个迭代算法，每一步都在修正猜测。
公式如下：
$$ O_{k+1}(x) = O_k(x) \cdot \left[ \frac{I(x)}{O_k(x) \otimes P(x)} \otimes P^*(x) \right] $$
让我们对应代码来翻译这个公式：$O_{k+1}$：新的猜测图像（更清晰）。$O_k$：当前的猜测图像。$I(x)$：我们拍到的模糊图像（data）。$P(x)$：点扩散函数（kernel，模糊核）。$\otimes$：卷积运算。
### 步骤详解
这是一个“猜——验证——修正”的过程：猜 (Estimate)：假设当前的图像 $O_k$ 就是真的物体。模糊 (Blur)：用已知的模糊核 $P$ 去模糊它，算出“如果我们猜得对，拍出来应该长什么样”：$$Reblurred = O_k \otimes P$$比较 (Ratio)：把“实际拍到的图”除以“算出来的模糊图”，得到一个“误差比率图”：$$Ratio = \frac{I}{Reblurred}$$如果猜对了，比率就是 1。如果猜小了，比率 $>1$；猜大了，比率 $<1$。修正 (Update)：把这个误差比率图，再通过模糊核“抹匀”（Back Projection，$\otimes P^*$），然后乘回原图中去修正它。为什么又要抹匀？因为误差不是在一个点产生的，而是由周围扩散过来的，所以要按原路分摊回去。代码中的实现Pythonrichardson_lucy(data, kernel, num_iter=iterations, clip=False)
scikit-image 库帮我们把上面那个复杂的迭代公式封装好了。iterations：迭代次数。次数越多，图像越锐利，但如果次数太多，会引入像斑点一样的“伪影”（Artifacts）。通常 10-20 次是比较安全的。
## 总结：这三者是如何协作的？
反卷积 1 (Pre-RL)：先用 rl_core 稍微把原始图片洗清楚一点。

傅里叶插值：用 fourier_interpolation 把网格加密（比如 100x100 变成 200x200），为提升分辨率提供“画布”。

计算累积量：用 cumulant 在这个大画布上，分析时间序列，把噪声洗掉，同时让光斑变小（分辨率提升 $\sqrt{2}$ 倍）。

反卷积 2 (Post-RL)：最后再用 rl_core 狠狠地锐化一次，得到最终的高清图。